In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from th2_data_services import Data
from datetime  import datetime
from typing import Dict
import os
import pickle
import json
import az_tree
import event_utils
import script_report_utils

## Data

In [ ]:
data_cache = Data.from_cache_file('./cache_2.5kk_events/cache_2.5kk_events.pickle')

In [ ]:
events = data_cache.limit(200_000)

In [ ]:
event = list(events.limit(1))[0]

In [ ]:
# messages = Data.from_cache_file('./messages.pickle')

In [ ]:
# # Build DS-core ETC (OPTIONAL (Will take few seconds to build))
# from th2_data_services.event_tree import EventTreeCollection
# from th2_data_services_lwdp.event_tree import HttpETCDriver
# 
# driver = HttpETCDriver()
# etc = EventTreeCollection(driver)
# etc.build(events)

In [ ]:
parent_events = event_utils.get_roots(events, 1000)

In [ ]:
parent = parent_events[0]

# Event Utils

### Get Type Totals

In [ ]:
type_totals = event_utils.get_type_totals(events=events)
type_totals

### Get Type Frequencies

In [ ]:
type_frequencies = event_utils.get_type_frequencies(
    events=events,
    types=['Service event', 'ModelMatrix'],
    aggregation_level="hours"
)
type_frequencies


### Get Category Totals

In [ ]:
category_totals = event_utils.get_category_totals(events=events,
                                                  categorizer=lambda event: event['type'])
category_totals


### Get Category Frequencies

In [ ]:
category_frequencies = event_utils.get_category_frequencies(
    events=events,
    categories=['Service event', 'ModelMatrix'],
    categorizer=lambda event: event['eventType']
)
category_frequencies

### Get Attached Message Ids

In [ ]:
attached_message_ids = event_utils.get_attached_message_ids(events=events)
attached_message_ids

### Get Attached Messages Totals

In [ ]:
attached_messages_totals = event_utils.get_attached_messages_totals(events=events)
attached_messages_totals

### Get Prior Parent Ids

In [ ]:
prior_parent_ids = event_utils.get_prior_parent_ids(events=events)
prior_parent_ids

### Get Attached Message Ids Index

In [ ]:
# Can taka a lot of time (~48sec)
attached_message_ids_index = event_utils.get_attached_message_ids_index(events=events)
attached_message_ids_index

### Get Roots

In [ ]:
roots = event_utils.get_roots(events=events, count=10)
roots

### Get Parents

In [ ]:
children_of_parent_1, parent_1 = event_utils.get_children_from_parent_id(events, parent['eventId'], 100)
parents = event_utils.get_parents(events=events, children=children_of_parent_1)
# or to find all parents
# parents = event_utils.get_parents(events=events, children=events)
assert [parent_1] == parents
parents

### Get Some

In [ ]:
some = event_utils.get_some(events=events, event_type='ModelCase', count=10)
some

### Get Events By Category

In [ ]:
events_by_category = event_utils.get_events_by_category(
    # events=events, category='ModelCase', count=10, categorizer=lambda event: event['eventType']
    # events=events, category="f0c3f6e8-fae7-40b6-8125-a16df981775d", count=10, categorizer=lambda event: event['parentEventId']
    events=events, category=False, count=10, categorizer=lambda event: event['successful']
)
events_by_category


### Get Related Events

In [ ]:
# # Needs events/messages from same time interval, otherwise will not match.
# related_events = event_utils.get_related_events(events=events, messages=messages, count=10)
# related_events

### Get Children From Parent Id

In [ ]:
children_from_parent_id = event_utils.get_children_from_parent_id(events=events,
                                                                  parent_id="f0c3f6e8-fae7-40b6-8125-a16df981775d",
                                                                  max_events=10)
children_from_parent_id


### Get Children From Parents

In [ ]:
children_from_parents, children_count = event_utils.get_children_from_parents(events=events,
                                                                              parents=parent_events,
                                                                              max_events=1)
print(f"Children count: {children_count}")
children_from_parents


### Get Children From Parents As List

In [ ]:
children_from_parents_as_list = event_utils.get_children_from_parents_as_list(events=events,
                                                                              parents=parent_events,
                                                                              max_events=1)
children_from_parents_as_list

### Sublist

In [ ]:
sublist = event_utils.sublist(
    events=events,
    start_time=datetime.fromisoformat("2022-03-16T10:50:16"),
    end_time=datetime.fromisoformat("2022-03-16T10:53:16")
)
sublist


### Build Roots Cache

In [ ]:
roots_cache = event_utils.build_roots_cache(events=events, depth=3, max_level=3)
roots_cache

### Extract Start Timestamp

In [ ]:
start_timestamp = event_utils.extract_start_timestamp(event=event)
start_timestamp

### Print Type Totals

In [ ]:
type_totals = event_utils.print_type_totals(
    events=events, 
    # return_html=True
)
type_totals

### Print Type Frequencies

In [ ]:
type_frequencies = event_utils.print_type_frequencies(
    events=events,
    event_types=['Service event', 'SendError','sendMessage'],
    aggregation_level='hours',
    # return_html=True
)
type_frequencies

### Print Category Totals

In [ ]:
category_totals = event_utils.print_category_totals(
    events=events, 
    # ignore_status=True
    categorizer=lambda event: event['eventType'],
    # # Presenting different possible use cases
    # categorizer=lambda event: event['eventName']
    # categorizer=lambda event: event['batchId']
)
category_totals

### Print Category Frequencies

In [ ]:
category_frequencies = event_utils.print_category_frequencies(
    events=events,
    event_types=["Service event", "ModelCase", "ModelMatrix"],
    categorizer=lambda event: event['eventType'],
    aggregation_level="hours",
    # return_html=True
)
category_frequencies


### Print Attached Messages Totals

In [ ]:
attached_messages_totals = event_utils.print_attached_messages_totals(events=events
                                                                      # return_html=True
                                                                      )
attached_messages_totals


### Print Roots

In [ ]:
roots = event_utils.print_roots(events=events, count=10, start=10)
roots

### Print Children

In [ ]:
children = event_utils.print_children(events=events,
                                      parent="d2e09664-a504-11ec-a6d5-2365793d9363",
                                      count=10,
                                      # verbose=False
                                      )
children


### Print Event

In [ ]:
event = event_utils.print_event(event=event)
event

### Print Some

In [ ]:
some = event_utils.print_some(
    events=events,
    event_type="ModelCase",
    count=10,
    # start=10,
    # failed=True,
    # verbose=False
)
some

### Print Some By Category

In [ ]:
some_by_category = event_utils.print_some_by_category(
    events=events,
    category="Service event",
    categorizer=lambda event: event['eventType'],
    count=10,
    # start=10,
    # failed=True
)
some_by_category


### Print Children From Parents

In [ ]:
children_from_parents = event_utils.print_children_from_parents(
    events=events,
    parents=parent_events,
    max_events=10
)
children_from_parents

### Print Children Stats From Parents

In [ ]:
children_stats_from_parents = event_utils.print_children_stats_from_parents(
    events=events,
    parents=parent_events,
    max_events=10,
    return_html=True
)
children_stats_from_parents

- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

# AZ Tree

### Get Event Tree From Parent Id

In [ ]:
get_event_tree_from_parent_id = az_tree.get_event_tree_from_parent_id(
    events=events,
    parent_id=parent['eventId'],
    depth=10,
    max_children=50,
    # body_to_simple_processors=None # TODO: Add examples
)
get_event_tree_from_parent_id


In [ ]:
# Export to json for better view
if not os.path.exists("./output"): os.mkdir("output")
with open("./output/event_tree_from_parent_id.json", "w") as f:
    json.dump(get_event_tree_from_parent_id, f, ensure_ascii=False, indent=2)

### Get Event Tree From Parent Events

In [ ]:
get_event_tree_from_parent_events = tree, index = az_tree.get_event_tree_from_parent_events(
    events=events,
    parents=parent_events,
    depth=10,
    max_children=50,
    # body_to_simple_processors=None # TODO: Add examples
)
get_event_tree_from_parent_events

In [ ]:
# Export to json for better view
if not os.path.exists("./output"): os.mkdir("output")
with open("./output/event_event_tree_from_parent_events_TREE.json", "w") as f:
    json.dump(tree, f, ensure_ascii=False, indent=2)

In [ ]:
# Export to json for better view
if not os.path.exists("./output"): os.mkdir("output")
with open("./output/event_event_tree_from_parent_events_INDEX.json", "w") as f:
    json.dump(index, f, ensure_ascii=False, indent=2)

### Save Tree As Json

In [ ]:
if not os.path.exists("./output"): os.mkdir("output")
if not os.path.exists("./output/save_tree"): os.mkdir("output/save_tree")
save_tree_as_json = az_tree.save_tree_as_json(
    tree=tree,
    json_file_path="./output/save_tree/save_tree.json",
    # file_categorizer=lambda key, leaf: key # Will create lots of json files!
)


### Transform Tree

In [ ]:
# # TODO: Add examples
# transform_tree = az_tree.transform_tree(events=events)

### Tree Walk

In [ ]:
tree_walk = az_tree.tree_walk(
    tree=tree,
    processor=lambda path, name, leaf: leaf.update({name: "/".join(path)}),
    tree_filter=lambda path, name, leaf: "[fail]" in name,
)
if not os.path.exists("./output"): os.mkdir("output")
with open("./output/tree_walk.json", "w") as f:
    json.dump(tree, f, ensure_ascii=False, indent=2)

### Tree Walk From Jsons

In [ ]:
tree_walk_from_jsons = az_tree.tree_walk_from_jsons(
    path_pattern="output/tree_walk.json",
    processor=lambda path, name, leaf: leaf.update({name: path[-1]}),
    tree_filter=lambda path, name, leaf: "[fail]" in name
)


### Tree Walk From Jsons (Scenario 2)

In [ ]:
results = {}
tree_walk_from_jsons = az_tree.tree_walk_from_jsons(
    path_pattern="output/tree_walk.json",
    processor=lambda path, name, leaf: results.update({path[-1]: leaf}),
    tree_filter=lambda path, name, leaf: "[fail]" in name
)
results


### Tree Update Totals

In [ ]:
# # TODO: Add example
# tree_update_totals = az_tree.tree_update_totals(...)

### Tree Get Category Totals

In [ ]:
tree_get_category_totals = az_tree.tree_get_category_totals(
    tree=tree,
    categorizer=lambda path, name, leaf: leaf['info']['type'] if 'info' in leaf else None,
    tree_filter=lambda path, name, leaf: "[fail]" not in name
)
tree_get_category_totals

### Tree Get Category Totals From Jsons

In [ ]:
# # TODO: Add example
tree_get_category_totals_from_jsons = az_tree.tree_get_category_totals_from_jsons(...)

### Search Tree

In [ ]:
# # TODO: Add example
search_tree = az_tree.search_tree(
    tree=tree,
    tree_filter=lambda path, name, leaf: "[fail]" not in name
)

### Search Tree From Jsons

In [ ]:
# # TODO: Add example
search_tree_from_jsons = az_tree.search_tree_from_jsons(...)

### Process Trees From Jsons

In [ ]:
# # TODO: Add example
process_trees_from_jsons = az_tree.process_trees_from_jsons(...)